In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/scriptpretrain-ngram/pretrain/train_bert.py
/kaggle/input/scriptpretrain-ngram/pretrain/NLP_Utils.py
/kaggle/input/scriptpretrain-ngram/pretrain/__init__.py
/kaggle/input/scriptpretrain-ngram/pretrain/transformers1/convert_pytorch_checkpoint_to_tf2.py
/kaggle/input/scriptpretrain-ngram/pretrain/transformers1/training_args_tf.py
/kaggle/input/scriptpretrain-ngram/pretrain/transformers1/tokenization_ctrl.py
/kaggle/input/scriptpretrain-ngram/pretrain/transformers1/modeling_utils.py
/kaggle/input/scriptpretrain-ngram/pretrain/transformers1/configuration_flaubert.py
/kaggle/input/scriptpretrain-ngram/pretrain/transformers1/modeling_roberta.py
/kaggle/input/scriptpretrain-ngram/pretrain/transformers1/hf_argparser.py
/kaggle/input/scriptpretrain-ngram/pretrain/transformers1/tokenization_auto.py
/kaggle/input/scriptpretrain-ngram/pretrain/transformers1/tokenization_reformer.py
/kaggle/input/scriptpretrain-ngram/pretrain/transformers1/trainer.py
/kaggle/input/scriptpretrain-ngram

In [2]:
import sys
sys.path.append('/kaggle/input/scriptpretrain-ngram/nezah_script_pretrain/pretrain') # 最后末尾不要加反斜杠

In [3]:
import random
import json
import transformers as _
from transformers1 import BertTokenizer
import torch
from torch.utils.data import Dataset,DataLoader
import numpy as np
from itertools import chain
import os

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [4]:
import random
import json
import transformers as _
from transformers1 import BertTokenizer
import torch
from torch.utils.data import Dataset,DataLoader
import numpy as np
from itertools import chain

def writeToJsonFile(path: str, obj):
    with open(path, "w", encoding="utf-8") as f:
        f.write(json.dumps(obj, ensure_ascii=False,indent=0))
def readFromJsonFile(path: str):
    with open(path, "r", encoding="utf-8") as f:
        return json.loads(f.read())

def loadData(path):
    allData=[]
    with open(path,"r") as f:
        for i in f:
            i=i.strip().split('\t')
            if len(i)==0:#防止空行
                break
            if len(i)==3:#训练集
                a,b,label=i
                a=a.split(' ')
                b=b.split(' ')
            else:#测试集，直接转为id形式
                a,b,label=i[0],i[1],-1
                a=a.split(' ')
                b=b.split(' ')
            allData.append([a,b,label])
    return allData

def calNegPos(ls):#计算正负比例
    posNum,negNum=0,0
    for i in ls:
        if i[2]==0:
            negNum+=1
        elif i[2]==1:
            posNum+=1
    posNum=1 if posNum==0 else posNum
    return negNum,posNum,round(negNum/posNum,4)

data_dir = '/kaggle/input/competition/challenge_all_data.txt'
allData=loadData(data_dir + '/challenge_all_data.txt')
random.shuffle(allData)

train_data=allData
valid_data=allData[-1000:]
print("训练集样本数量：", len(train_data))

def paddingList(ls:list,val,returnTensor=False):
    ls=ls[:]#不要改变了原list尺寸
    maxLen=max([len(i) for i in ls])
    for i in range(len(ls)):
        ls[i]=ls[i]+[val]*(maxLen-len(ls[i]))
    return torch.tensor(ls,device='cuda') if returnTensor else ls

def truncate(a:list,b:list,maxLen):
    maxLen-=3#空留给cls sep sep
    assert maxLen>=0
    len2=maxLen//2#若为奇数，更长部分给左边
    len1=maxLen-len2
    #一共就a超长与否，b超长与否，组合的四种情况
    if len(a)+len(b)>maxLen:#需要截断
        if len(a)<=len1 and len(b)>len2:
            b=b[:maxLen-len(a)]
        elif len(a)>len1 and len(b)<=len2:
            a=a[:maxLen-len(b)]
        elif len(a)>len1 and len(b)>len2:
            a=a[:len1]
            b=b[:len2]
    return a,b

class MLM_Data(Dataset):
    #传入句子对列表
    def __init__(self,textLs:list,maxLen:int,tk:BertTokenizer):
        super().__init__()
        self.data=textLs
        self.maxLen=maxLen
        self.tk=tk
        self.spNum=len(tk.all_special_tokens)
        self.tkNum=tk.vocab_size

    def __len__(self):
        return len(self.data)

    def random_mask(self,text_ids):
        input_ids, output_ids = [], []
        rands = np.random.random(len(text_ids))
        idx=0
        while idx<len(rands):
            if rands[idx]<0.15:#需要mask
                ngram=np.random.choice([1,2,3], p=[0.7,0.2,0.1])#若要mask，进行x_gram mask的概率
                if ngram==3 and len(rands)<7:#太大的gram不要应用于过短文本
                    ngram=2
                if ngram==2 and len(rands)<4:
                    ngram=1
                L=idx+1
                R=idx+ngram#最终需要mask的右边界（开）
                while L<R and L<len(rands):
                    rands[L]=np.random.random()*0.15#强制mask
                    L+=1
                idx=R
                if idx<len(rands):
                    rands[idx]=1#禁止mask片段的下一个token被mask，防止一大片连续mask
            idx+=1

        for r, i in zip(rands, text_ids):
            if r < 0.15 * 0.8:
                input_ids.append(self.tk.mask_token_id)
                output_ids.append(i)#mask预测自己
            elif r < 0.15 * 0.9:
                input_ids.append(i)
                output_ids.append(i)#自己预测自己
            elif r < 0.15:
                input_ids.append(np.random.randint(self.spNum,self.tkNum))
                output_ids.append(i)#随机的一个词预测自己，随机词不会从特殊符号中选取，有小概率抽到自己
            else:
                input_ids.append(i)
                output_ids.append(-100)#保持原样不预测

        return input_ids, output_ids

    #耗时操作在此进行，可用上多进程
    def __getitem__(self, item):
        text1,text2,_=self.data[item]#预处理，mask等操作
        if random.random()>0.5:
            text1,text2=text2,text1#交换位置
        text1,text2=truncate(text1,text2,self.maxLen)
        text1_ids,text2_ids = self.tk.convert_tokens_to_ids(text1),self.tk.convert_tokens_to_ids(text2)
        text1_ids, out1_ids = self.random_mask(text1_ids)#添加mask预测
        text2_ids, out2_ids = self.random_mask(text2_ids)
        input_ids = [self.tk.cls_token_id] + text1_ids + [self.tk.sep_token_id] + text2_ids + [self.tk.sep_token_id]#拼接
        token_type_ids=[0]*(len(text1_ids)+2)+[1]*(len(text2_ids)+1)
        labels = [-100] + out1_ids + [-100] + out2_ids + [-100]
        assert len(input_ids)==len(token_type_ids)==len(labels)
        return {'input_ids':input_ids,'token_type_ids':token_type_ids,'labels':labels}

    @classmethod
    def collate(cls,batch):
        input_ids=[i['input_ids'] for i in batch]
        token_type_ids=[i['token_type_ids'] for i in batch]
        labels=[i['labels'] for i in batch]
        input_ids=paddingList(input_ids,0,returnTensor=True)
        token_type_ids=paddingList(token_type_ids,0,returnTensor=True)
        labels=paddingList(labels,-100,returnTensor=True)
        attention_mask=(input_ids!=0)
        return {'input_ids':input_ids,'token_type_ids':token_type_ids
                ,'attention_mask':attention_mask,'labels':labels}




unionList=lambda ls:list(chain(*ls))#按元素拼接
splitList=lambda x,bs:[x[i:i+bs] for i in range(0,len(x),bs)]#按bs切分


#sortBsNum：原序列按多少个bs块为单位排序，可用来增强随机性
#比如如果每次打乱后都全体一起排序，那每次都是一样的
def blockShuffle(data:list,bs:int,sortBsNum,key):
    random.shuffle(data)#先打乱
    tail=len(data)%bs#计算碎片长度
    tail=[] if tail==0 else data[-tail:]
    data=data[:len(data)-len(tail)]
    assert len(data)%bs==0#剩下的一定能被bs整除
    sortBsNum=len(data)//bs if sortBsNum is None else sortBsNum#为None就是整体排序
    data=splitList(data,sortBsNum*bs)
    data=[sorted(i,key=key,reverse=True) for i in data]#每个大块进行降排序
    data=unionList(data)
    data=splitList(data,bs)#最后，按bs分块
    random.shuffle(data)#块间打乱
    data=unionList(data)+tail
    return data
from torch.utils.data.dataloader import _SingleProcessDataLoaderIter,_MultiProcessingDataLoaderIter
#每轮迭代重新分块shuffle数据的DataLoader
class blockShuffleDataLoader(DataLoader):
    def __init__(self, dataset: Dataset,sortBsNum,key,**kwargs):
        assert isinstance(dataset.data,list)#需要有list类型的data属性
        super().__init__(dataset,**kwargs)#父类的参数传过去
        self.sortBsNum=sortBsNum
        self.key=key

    def __iter__(self):
        #分块shuffle
        self.dataset.data=blockShuffle(self.dataset.data,self.batch_size,self.sortBsNum,self.key)
        if self.num_workers == 0:
            return _SingleProcessDataLoaderIter(self)
        else:
            return _MultiProcessingDataLoaderIter(self)


训练集样本数量： 70001


In [5]:
# coding:utf-8
import numpy as np
import random
import os
random.seed(0)
np.random.seed(0)#seed应该在main里尽早设置，以防万一
os.environ['PYTHONHASHSEED'] =str(0)#消除hash算法的随机性
import transformers as _
from transformers1 import Trainer, TrainingArguments,BertTokenizer
#from NLP_Utils import MLM_Data,train_data,blockShuffleDataLoader

from NEZHA.configuration_nezha import NeZhaConfig
from NEZHA.modeling_nezha import NeZhaForMaskedLM

maxlen=32
batch_size=128
#vocab_file_dir = './nezha_model/vocab.txt'
bert_dir = '/kaggle/input/pretrainbertmodel/nezha-cn-base_new/nezha-cn-base/'
vocab_file_dir='/kaggle/input/pretrainbertmodel/nezha-cn-base_new/nezha-cn-base/vocab.txt'
tokenizer = BertTokenizer.from_pretrained(vocab_file_dir)

config = NeZhaConfig(
    vocab_size=len(tokenizer),
    hidden_size=768,
    num_hidden_layers=12,
    num_attention_heads=12,
    max_position_embeddings=512,
)


#model = NeZhaForMaskedLM.from_pretrained("../../nezha-cn-base/")
model = NeZhaForMaskedLM.from_pretrained(bert_dir)

model.resize_token_embeddings(len(tokenizer))
train_MLM_data=MLM_Data(train_data,maxlen,tokenizer)
#自己定义dataloader，不要用huggingface的
dl=blockShuffleDataLoader(train_MLM_data,None,key=lambda x:len(x[0])+len(x[1]),shuffle=False
                          ,batch_size=batch_size,collate_fn=train_MLM_data.collate)

training_args = TrainingArguments(
    output_dir='./nezha_output',
    overwrite_output_dir=True,
    num_train_epochs=400,
    per_device_train_batch_size=batch_size,
    save_steps=len(dl)*10000,#每10个epoch save一次
    save_total_limit=3,
    logging_steps=len(dl),#每个epoch log一次
    seed=2021,
    learning_rate=5e-5,
    weight_decay=0.01,
    warmup_steps=int(450000*150/batch_size*0.03)
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataLoader=dl,
    prediction_loss_only=True,
)

if __name__ == '__main__':
    trainer.train()
    trainer.save_model('./nezha_model')


Some weights of the model checkpoint at /kaggle/input/pretrainbertmodel/nezha-cn-base_new/nezha-cn-base/ were not used when initializing NeZhaForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing NeZhaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NeZhaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/opt/conda/lib/python3.7/site-packages/transformers/modeling_utils.py:813: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  "The `device` argument is deprecated and will be removed in v5 of Transformers.", FutureWarning



{"loss": 0.44194976579368334, "learning_rate": 1.7288242730720608e-06, "epoch": 1.0, "step": 547}

预训练第0轮耗时： 61.71517848968506

{"loss": 0.0001675276073911352, "learning_rate": 3.4576485461441217e-06, "epoch": 2.0, "step": 1094}

预训练第1轮耗时： 60.227346420288086

{"loss": 1.2032099880218892e-06, "learning_rate": 5.186472819216182e-06, "epoch": 3.0, "step": 1641}

预训练第2轮耗时： 60.28002977371216

{"loss": 5.702512383728453e-07, "learning_rate": 6.915297092288243e-06, "epoch": 4.0, "step": 2188}

预训练第3轮耗时： 60.19330620765686

{"loss": 1.6582422256534637e-07, "learning_rate": 8.644121365360304e-06, "epoch": 5.0, "step": 2735}

预训练第4轮耗时： 60.29735732078552

{"loss": 2.703041681419234e-06, "learning_rate": 1.0372945638432365e-05, "epoch": 6.0, "step": 3282}

预训练第5轮耗时： 60.15055584907532

{"loss": 1.3903436257256358e-09, "learning_rate": 1.2101769911504425e-05, "epoch": 7.0, "step": 3829}

预训练第6轮耗时： 60.414249897003174

{"loss": 6.639828317474062e-10, "learning_rate": 1.3830594184576487e-05, "epoch": 8